In [3]:
import csv

input_csv_path = r"C:\Users\SAADB\Desktop\Python_Code\Google_Hackathon\ossl_cleaned.csv"

output_csv_path = r"C:\Users\SAADB\Desktop\Python_Code\Google_Hackathon\ossl_cleaned_head10.csv"

with open(input_csv_path, "r", newline="", encoding="utf-8") as infile, \
     open(output_csv_path, "w", newline="", encoding="utf-8") as outfile:

    reader = csv.reader(infile)
    writer = csv.writer(outfile)

    header = next(reader)
    writer.writerow(header)

    for i, row in enumerate(reader):
        if i >= 10:
            break
        writer.writerow(row)

print("Saved first 10 rows to:", output_csv_path)


Saved first 10 rows to: C:\Users\SAADB\Desktop\Python_Code\Google_Hackathon\ossl_cleaned_head10.csv


In [5]:
import pandas as pd

orig_path = input_csv_path
clean_path = output_csv_path

# adjust this depending on your actual column naming convention
# Example assumes columns are like: scan_visnir.350_ref ... scan_visnir.2500_ref
def get_visnir_cols(cols):
    visnir = []
    for c in cols:
        s = str(c)
        # pick a pattern that matches your actual column names
        # Option A: "scan_visnir.350_ref"
        if s.startswith("scan_visnir.") and s.endswith("_ref"):
            try:
                wl = float(s.split(".")[1].split("_")[0])
                if 350 <= wl <= 2500:
                    visnir.append(c)
            except:
                pass
        # Option B (if names are like "scan_visnir350")
        elif s.startswith("scan_visnir"):
            tail = s.replace("scan_visnir", "")
            if tail.isdigit():
                wl = int(tail)
                if 350 <= wl <= 2500:
                    visnir.append(c)
    return sorted(visnir)

def summarize(path, label):
    df = pd.read_csv(path, nrows=5)  # just to get columns fast
    visnir_cols = get_visnir_cols(df.columns)

    print(f"\n[{label}] VISNIR col count:", len(visnir_cols))
    if not visnir_cols:
        print("No VISNIR columns matched your naming pattern. Print a few columns to adjust the matcher.")
        print("First 30 columns:", list(df.columns)[:30])
        return

    # Now actually load only those columns (fast-ish, still reads file but not all columns)
    dfv = pd.read_csv(path, usecols=visnir_cols)

    # % rows where ALL visnir columns are missing
    all_missing_rows = dfv.isna().all(axis=1).mean() * 100

    # % cells missing overall in visnir block
    overall_missing = dfv.isna().mean().mean() * 100

    # check if any column is entirely empty
    fully_empty_cols = dfv.columns[dfv.isna().all(axis=0)].tolist()

    print(f"[{label}] % rows with NO visnir spectrum at all: {all_missing_rows:.2f}%")
    print(f"[{label}] overall missingness in visnir block: {overall_missing:.2f}%")
    print(f"[{label}] fully-empty visnir columns: {len(fully_empty_cols)}")
    if fully_empty_cols[:10]:
        print("  examples:", fully_empty_cols[:10])

summarize(orig_path, "ORIGINAL")
summarize(clean_path, "CLEANED")



[ORIGINAL] VISNIR col count: 1076
[ORIGINAL] % rows with NO visnir spectrum at all: 52.35%
[ORIGINAL] overall missingness in visnir block: 53.04%
[ORIGINAL] fully-empty visnir columns: 0

[CLEANED] VISNIR col count: 1076
[CLEANED] % rows with NO visnir spectrum at all: 100.00%
[CLEANED] overall missingness in visnir block: 100.00%
[CLEANED] fully-empty visnir columns: 1076
  examples: ['scan_visnir.350_ref', 'scan_visnir.352_ref', 'scan_visnir.354_ref', 'scan_visnir.356_ref', 'scan_visnir.358_ref', 'scan_visnir.360_ref', 'scan_visnir.362_ref', 'scan_visnir.364_ref', 'scan_visnir.366_ref', 'scan_visnir.368_ref']


In [6]:
import pandas as pd

orig_path  = input_csv_path
clean_path = output_csv_path

def visnir_health(path, label):
    cols = pd.read_csv(path, nrows=1).columns
    visnir_cols = [c for c in cols if c.startswith("scan_visnir.") and c.endswith("_ref")]

    dfv = pd.read_csv(path, usecols=visnir_cols)
    rows_all_missing = dfv.isna().all(axis=1).mean() * 100
    overall_missing  = dfv.isna().mean().mean() * 100
    fully_empty_cols = dfv.columns[dfv.isna().all(axis=0)].size

    print(f"[{label}] visnir_cols={len(visnir_cols)} | rows_all_missing={rows_all_missing:.2f}% "
          f"| overall_missing={overall_missing:.2f}% | fully_empty_cols={fully_empty_cols}")

visnir_health(orig_path,  "ORIGINAL")
visnir_health(clean_path, "CLEANED")


[ORIGINAL] visnir_cols=1076 | rows_all_missing=52.35% | overall_missing=53.04% | fully_empty_cols=0
[CLEANED] visnir_cols=1076 | rows_all_missing=100.00% | overall_missing=100.00% | fully_empty_cols=1076


In [7]:
import pandas as pd

input_csv_path  = input_csv_path   # your cleaned/full CSV
output_csv_path = output_csv_path  # where you want the filtered CSV

# Read just the header to find VISNIR columns
cols = pd.read_csv(input_csv_path, nrows=1).columns
visnir_cols = [c for c in cols if c.startswith("scan_visnir.") and c.endswith("_ref")]

# Load full dataset
df = pd.read_csv(input_csv_path)

# Keep rows where at least ONE VISNIR wavelength exists
mask_has_visnir = df[visnir_cols].notna().any(axis=1)
df_visnir_only = df.loc[mask_has_visnir]

print(f"Rows before: {len(df)}")
print(f"Rows with VISNIR: {len(df_visnir_only)}")

# Save
df_visnir_only.to_csv(output_csv_path, index=False)
print("Saved VISNIR-only dataset to:", output_csv_path)


C:\Users\SAADB\AppData\Local\Temp\ipykernel_41424\2401201479.py:11: DtypeWarning: Columns (0: location.country_iso.3166_txt, 1: efferv_usda.a479_class, 2: scan.visnir.date.begin_iso.8601_yyyy.mm.dd, 3: scan.visnir.date.end_iso.8601_yyyy.mm.dd, 4: scan.mir.date.begin_iso.8601_yyyy.mm.dd, 5: scan.mir.date.end_iso.8601_yyyy.mm.dd) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(input_csv_path)


Rows before: 135651
Rows with VISNIR: 64644


KeyboardInterrupt: 

In [8]:
import pandas as pd
import numpy as np

input_csv_path = r"C:\Users\SAADB\Desktop\Python_Code\Google_Hackathon\Model_and_Results\ossl_test_union.csv"      # test union CSV
output_csv_path = r"C:\Users\SAADB\Desktop\Python_Code\Google_Hackathon\Model_and_Results\ossl_sample.csv"    # where to save the sample

# Load data
df = pd.read_csv(input_csv_path)

# Identify VISNIR columns
visnir_cols = [
    c for c in df.columns
    if c.startswith("scan_visnir.") and c.endswith("_ref")
]

# Split into VISNIR-present vs VISNIR-missing
df_has_visnir = df[df[visnir_cols].notna().any(axis=1)]
df_no_visnir  = df[df[visnir_cols].isna().all(axis=1)]

# Sanity check
if len(df_has_visnir) < 10:
    raise ValueError("Not enough rows with VISNIR to sample 10.")

# Sample
sample_visnir = df_has_visnir.sample(n=10, random_state=42)
sample_other  = df_no_visnir.sample(n=10, random_state=42)

# Combine + shuffle
sample_20 = pd.concat([sample_visnir, sample_other], axis=0)
sample_20 = sample_20.sample(frac=1, random_state=42).reset_index(drop=True)

# Save
sample_20.to_csv(output_csv_path, index=False)

print("Saved sample of 20 rows")
print("Rows with VISNIR:", sample_20[visnir_cols].notna().any(axis=1).sum())


C:\Users\SAADB\AppData\Local\Temp\ipykernel_41424\1765682829.py:8: DtypeWarning: Columns (0: location.country_iso.3166_txt, 1: efferv_usda.a479_class, 2: scan.visnir.date.begin_iso.8601_yyyy.mm.dd, 3: scan.visnir.date.end_iso.8601_yyyy.mm.dd, 4: scan.mir.date.begin_iso.8601_yyyy.mm.dd, 5: scan.mir.date.end_iso.8601_yyyy.mm.dd) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(input_csv_path)


Saved sample of 20 rows
Rows with VISNIR: 10


In [ ]:
import pandas as pd

input_csv_path = input_csv_path
output_csv_path = output_csv_path

df_head20 = pd.read_csv(input_csv_path, nrows=20)
df_head20.to_csv(output_csv_path, index=False)

print("Saved first 20 rows")
